# W01A — Introducción (DDIA Cap. 1) + Entorno local + Evidencia

## Qué haremos hoy
- Entender el “por qué” (DDIA Cap. 1): **Reliability / Scalability / Maintainability**
- Montar un entorno local reproducible (sin Docker).
- Usar SQL solo como **instrumento de evidencia** (no como tema formal aún).

## Mapa del curso (conceptual)
- **Bronze/Raw**: dato tal cual llega (trazabilidad)
- **Silver**: limpieza + tipado + reglas de calidad
- **Gold**: modelado para consulta (hechos/dimensiones/marts/métricas)

## Relación con carpetas (desde ya)
- `data/raw/`  → Bronze/Raw (archivos originales)
- `data/silver/` → outputs intermedios (Parquet/tablas exportadas)
- `data/gold/` → marts/métricas/exports listos para consumo
- `data/exoplanets.duckdb` → “warehouse local” (tablas)
- `artifacts/` → evidencia (JSON de tests, profiling, planes, logs)
- `docs/` → decisiones, runbook, glosario (mantenibilidad)


In [1]:
import os
os.getcwd()

'/Users/itnas/Documents/UQ/FisicaComputacional3/Estudiantes/notebooks'

In [2]:
# 1) Setup (cross-platform)
import sys, platform, hashlib
from pathlib import Path
import duckdb
import os

os.chdir("..")

PROJECT_ROOT = Path(".").resolve()

DATA_DIR = PROJECT_ROOT / "data"
RAW_DIR  = DATA_DIR / "raw"
SILVER_DIR = DATA_DIR / "silver"
GOLD_DIR = DATA_DIR / "gold"
ARTIFACTS_DIR = PROJECT_ROOT / "artifacts"
DOCS_DIR = PROJECT_ROOT / "docs"

for d in [RAW_DIR, SILVER_DIR, GOLD_DIR, ARTIFACTS_DIR, DOCS_DIR]:
    d.mkdir(parents=True, exist_ok=True)

DB_PATH = DATA_DIR / "exoplanets.duckdb"
con = duckdb.connect(str(DB_PATH))

def show(path: Path):
    return {"exists": path.exists(), "path": str(path), "size_bytes": path.stat().st_size if path.exists() else None}


print("OS:", platform.platform())
print("Python:", sys.version.split()[0])
print("DuckDB:", con.execute("SELECT version()").fetchone()[0])
show(DB_PATH)


OS: macOS-26.0.1-arm64-arm-64bit
Python: 3.9.6
DuckDB: v1.4.4


{'exists': True,
 'path': '/Users/itnas/Documents/UQ/FisicaComputacional3/Estudiantes/data/exoplanets.duckdb',
 'size_bytes': 12288}

## Evidencia mínima: ¿el motor responde?
Esto NO es “clase de SQL”. Es verificar que el motor funciona.

In [3]:
con.execute("SELECT 42 AS answer").fetchall()

[(42,)]

## “Experimento” controlado: una tabla pequeña
La idea: **consulta → evidencia**.

In [4]:
con.execute("CREATE OR REPLACE TABLE demo_numbers(x INTEGER)")
con.execute("INSERT INTO demo_numbers VALUES (1), (2), (3)")
con.execute("SELECT SUM(x) AS s FROM demo_numbers").fetchall()


[(6,)]

## Tu turno (en clase): 2 mini-tablas + 2 métricas
Ejecuta y pega los outputs al final del notebook.

In [5]:
# TU TURNO 1
con.execute("CREATE OR REPLACE TABLE students(name VARCHAR, semester INTEGER)")
con.execute("INSERT INTO students VALUES ('Ana', 7), ('Luis', 8), ('Sofia', 10)")
con.execute("SELECT semester, COUNT(*) n FROM students GROUP BY 1 ORDER BY 1").fetchall()


[(7, 1), (8, 1), (10, 1)]

In [6]:
# TU TURNO 2
con.execute("CREATE OR REPLACE TABLE submissions(name VARCHAR, lab VARCHAR, ok BOOLEAN)")
con.execute("""
INSERT INTO submissions VALUES
  ('Ana', 'W01', TRUE),
  ('Ana', 'W02', FALSE),
  ('Luis','W01', TRUE),
  ('Luis','W02', TRUE),
  ('Sofia','W01', TRUE)
""")
con.execute("SELECT name, COUNT(*) n FROM submissions GROUP BY 1 ORDER BY n DESC").fetchall()


[('Luis', 2), ('Ana', 2), ('Sofia', 1)]

In [7]:
try:
    con.close()
    print("DuckDB connection closed.")
except NameError:
    print("No connection named 'con' in this notebook.")


DuckDB connection closed.


## Entregable W01A (en clase)
Crea estos archivos (usa `docs/templates/` si existe):
- `docs/decisions_log.md`
- `docs/glossary.md`
- `docs/w01a_run.md`


## Reflexión (DDIA Cap.1)
- ¿Qué haría que tu sistema sea “no confiable” aunque el código corra?
- ¿Qué documento te gustaría encontrar si heredas el proyecto de otro equipo?
